# Практичне зайняття 3. Двословний індекс і координатний інвертований індекс


* Побудувати двословний індекс і координатний інвертований індекс по колекції документів.
* Реалізувати фразовий пошук та пошук з урахуванням відстані для кожного з них.

### Ініціалізуємо бібліотеку NLTK

In [1]:
from info_search.lab_1.nltk_utils import init_nltk

init_nltk()

[nltk_data] Downloading package punkt to /home/o.kyba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Двословний індекс

Для виконання завадння з побудови двослівного індексу було написано нову функцію для парсингу термінів, щоб фона повертала термін, який склдається з двох слів (`info_search.lab_3.two_words_index.parse_terms_for_2_words_index`). Також було прийнято рішення відмовитися від стоп слів, щоб коректно реалізувати фразовий пошук.

### Побудуємо лекскон та двослівний індекс

За рахунок того, що тепер у нас термін складється з двох слів, ми можемо перевикористати клас однослівного інвертованого індексу для поббулови двослівного.

In [2]:
from pymorphy3 import MorphAnalyzer

from info_search.lab_1.lexicon import Lexicon
from info_search.lab_1.nltk_utils import init_nltk
from info_search.lab_1.preprocessing import WordsNormalizer, WordsTokenizer
from info_search.lab_1.readers import EpubReader
from info_search.lab_2.inverted_index import (
    InvertedIndexBuilder,
    InvertedIndexQueryExecutor,
)

from info_search.lab_3.two_words_index import parse_terms_for_2_words_index

books = (
    "books/451-za-farengeytom.epub",
    "books/1984.epub",
    "books/atlant-rozpraviv-pliechi-1.epub",
    "books/atlant-rozpraviv-pliechi-2.epub",
    "books/atlant-rozpraviv-pliechi-3.epub",
    "books/haksli-oldos-prekrasnyy-novyy-svit.epub",
    "books/kulbabove-vino.epub",
    "books/na-zakhidnomu-fronti-bez-zmin.epub",
    "books/proshchavai-zbroie.epub",
    "books/sapients-istoriya-lyudstva.epub",
)

reader = EpubReader(books)
tokenaizer = WordsTokenizer(stop_words=[])
normalizer = WordsNormalizer(MorphAnalyzer(lang="uk"))
lexicon = Lexicon()
two_words_idx_builder = InvertedIndexBuilder()

for doc_name, term in parse_terms_for_2_words_index(reader, tokenaizer, normalizer):
    doc_id, term_id = lexicon.add_term(doc_name, term)
    two_words_idx_builder.add_term(doc_id, term_id)

two_words_idx = two_words_idx_builder.build()

  0%|          | 0/10 [00:00<?, ?it/s]/home/o.kyba/PycharmProjects/info_search/venv/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
100%|██████████| 10/10 [00:57<00:00,  5.71s/it]


### Фразовий пошук

За допомгою двослівного індексу, було реалізовано фразовий пошук. У класі `info_search.lab_3.two_words_index.TwoWordsIndexQueryExecutor` виконується препроцесінг запиту, а саме - розбиття запиту на двослівні терміни. Шукаються всі документи для цих термінів, а на постобробці - ці результати об'єднуються за схожим алгортимом, що був використаний для реалізації оператору `AND` в булевому пошуку.

In [7]:
from info_search.lab_3.two_words_index import TwoWordsIndexQueryExecutor

executor = TwoWordsIndexQueryExecutor(two_words_idx, lexicon, tokenaizer, normalizer)

docs_1 = executor.search("Даґні Таґґарт")
docs_2 = executor.search("міс Таґґарт")
docs_3 = executor.search("тримається на самих лише м’язах")

print(f"Документи, де є 'Даґні Таґґарт': {docs_1}\n")
print(f"Документи, де є 'міс Таґґарт': {docs_2}\n")
print(f"Документи, де є 'світ тримається на самих лише м’язах': {docs_3}\n")

Документи, де є 'Даґні Таґґарт': ['atlant-rozpraviv-pliechi-1.epub/ch1-13.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-15.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-12.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-18.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-17.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-20.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-3.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-5.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-2.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-9.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-16.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-13.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-27.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-26.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-8.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-5.xhtml', 'atlant-rozpra

## Координатний індекс